In [ ]:
!rm -rf pendulum
!git clone https://github.com/rland93/pendulum.git
!pip install ipympl

Set system path so that pendulum module can be found

In [2]:
import sys
sys.path.append('pendulum/')
print(sys.executable)
print(sys.version)

/home/ms/prg/pendulum/venv/bin/python
3.8.8 (default, Apr 13 2021, 19:58:26) 
[GCC 7.3.0]


Import Packages

In [5]:
import numpy as np
from pendulum import controller, pendulum, sim, utils
from pendulum.viz import Visualizer
from IPython.display import HTML
import matplotlib.pyplot as plt
%matplotlib inline

ModuleNotFoundError: No module named 'pendulum'

We set the timestep of the simulation (`dt`), the total simulation time (`t_final`), and create a pendulum. We set the pendulum to be in the upright position: 

$$ [x, \dot{x}, \theta, \dot{\theta}] = [0,0,0,0]$$

In [ ]:
dt = 0.01
t_final = 15
pend = pendulum.Pendulum(2.0, 3.0, 2.0, initial_state=np.array([0,0,-.05,0]))

Forcing function

In [ ]:
c4 = 2, 3.0, 7, 2
fshift = 6
force_fn = lambda t: c1 * np.sin(c2*t) * c3/(c4*np.sqrt(np.pi)) * np.exp(-((t-fshift)/c4)**2)
fx = np.linspace(0,t_final, 500)
fy = force_fn(fx)
ax.plot(fx, fy)

Simulation definition

In [ ]:
simulation = sim.Simulation(dt, t_final, force_fn)

We create a new LQR controller. $Q$ has the weighted cost function over the inputs; here, we penalize error in $\theta$ highest. We also want to penalize control input, becuase it's assumed that we have finite actuation.

In [ ]:
Q = [0,0,1,0]
R = 0.005

cont = controller.LQR(pend, dt, 6, Q, R)

Run the simulation.

In [ ]:
results = simulation.simulate(pend, cont, plot=False)

View the results

In [ ]:
fig, ax = plt.subplots()
for s in results['state']:
    ax.plot(results[('state', s)], label=s)
    ax.legend()

In [ ]:
fig, ax = plt.subplots()
ax.plot(results['control action'], label='control action')
ax.plot(results['forces'], label='force')
ax.plot(results[('state', 't')], label='theta')
ax.legend()

In [ ]:
visualizer = Visualizer(results, pend, speed=4)
anim = visualizer.animate((9.5, 5))

In [ ]:
HTML(anim.to_html5_video())